In [1]:
from datetime import date

import flights
from flights import get_system_days
import queue
import threading

from multiprocessing import Pool
pool = Pool(processes=2)

In [2]:
import csv
AIRPORTS = []



with open('airports.csv', 'r', encoding="utf8") as filename:
    reader = (line for line in csv.reader(filename))
    next(reader)
    for row in reader:
        if row[7] == 'EU':
            if row[2] == 'large_airport' and row[11] == 'yes':
                AIRPORTS.append(row)
            elif row[2] == 'medium_airport' and row[9] in ['FR-N', 'FR-B']and row[11] == 'yes':
                AIRPORTS.append(row)
            elif row[8] == 'GI'and row[11] == 'yes':
                AIRPORTS.append(row)

In [3]:

    
# http://www.madhur.co.in/blog/2015/11/02/countdownlatch-python.html    
class CountDownLatch(object):
    def __init__(self, count=1):
        self.count = count
        self.lock = threading.Condition()

    def countdown(self):
        self.lock.acquire()
        self.count -= 1
        if self.count <= 0:
            self.lock.notifyAll()
        self.lock.release()

    def wait(self):
        self.lock.acquire()
        while self.count > 0:
            self.lock.wait()
        self.lock.release()

In [4]:
from py2neo import authenticate, Graph, Relationship
graph = Graph("http://localhost:7474/db/data/")
#graph.delete_all()
print ("connected")
from py2neo import Node, Relationship

connected


In [5]:
def create(graph, thing):
    t = threading.Thread(target=lambda x,y: x.merge(y), args=(graph,thing))
    t.setDaemon(True)
    t.start()
    

from datetime import datetime
from datetime import timedelta
start_date = datetime(2017,1,1)

print (start_date, start_date+timedelta(days=1))

2017-01-01 00:00:00 2017-01-02 00:00:00


In [ ]:

"""daysq = queue.Queue()
latch = CountDownLatch(NO_OF_DAYS)

def create_day(index, daysq, latch):
    day = Node("Date", date = index)
    graph.create( day )
    daysq.put(day)
    if index % 7 == 0:
        print(index)
    latch.countdown()

threads = []
for index in range(NO_OF_DAYS):
    t = threading.Thread(target=create_day, args=(index,daysq, latch))
    t.start()
    
latch.wait()
    
days = []

while not daysq.empty():
    days.append(daysq.get())"""

In [6]:
regions, countries, continents, cities = set(), set(), set(), set()

for row in AIRPORTS:
    regions.add(row[9])
    countries.add(row[8])
    continents.add(row[7])
    cities.add(row[10])

print(len(cities), len(AIRPORTS))

149 161


In [7]:
def create_area_node(name, graph, value,latch,output):
    print(", ".join([name, value]))
    kw = {name.lower() : value}
    node = Node(name.title(), **kw)
    create(graph, node)
    output[value] = node
    latch.countdown()
    
r,c,co,ci = {},{},{},{}
area_nodes = {}


    
for name, dataset, output in [("Regions", regions,r), ("Countries", countries, c),
                              ("Continents", continents, co), ("Cities", cities, ci)]:
    latch = CountDownLatch(len(dataset))
    for row in dataset:
        
        threading.Thread(target= create_area_node, args=(name, graph, row,latch, output)).start()
        
    latch.wait()

Regions, GB-NIR
Regions, CH-GE
Regions, UA-14
Regions, NO-19
Regions, IT-21
Regions, RU-KDA
Regions, IE-CE
Regions, SI-061
Regions, IT-88
Regions, IT-42
Regions, IT-45
Regions, DE-BY
Regions, RU-TA
Regions, DE-BE
Regions, FR-A
Regions, DE-BR
Regions, IT-62
Regions, HU-PE
Regions, IT-72
Regions, DE-HB
Regions, ES-V
Regions, RU-MOS
Regions, DE-HE
Regions, RU-BA
Regions, ES-GA
Regions, RU-SPE
Regions, GR-A1
Regions, PL-WP
Regions, ME-16
Regions, DE-SN
Regions, NL-NB
Regions, DE-NW
Regions, LV-RIX
Regions, ES-M
Regions, IT-82
Regions, ES-CN
Regions, IE-C
Regions, IT-75
Regions, CH-ZH
Regions, ES-CT
Regions, PL-MA
Regions, UA-51
Regions, IT-25
Regions, NO-17
Regions, NO-12
Regions, LU-L
Regions, NO-18
Regions, GR-91
Regions, ES-PM
Regions, PT-20
Regions, FR-B
Regions, IS-2
Regions, BG-23
Regions, SE-BD
Regions, PL-DS
Regions, ES-AN
Regions, DE-BW
Regions, DK-81
Regions, XK-01
Regions, DE-NI
Regions, GB-U-A
Regions, DK-84
Regions, GB-ENG
Regions, DK-83
Regions, AL-11
Regions, UA-63
Regions, 

Exception in thread Thread-528:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\packages\neo4j\v1\session.py", line 124, in session
    session = self.session_pool.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-a42a1bd8a418>", line 2, in <lambda>
    t = threading.Thread(target=lambda x,y: x.merge(y), args=(graph,thing))
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\database\__init__.py", line 639, in merge
    self.begin(autocommit=True).merge(subgraph, label, *property_keys)
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\database\__init__.py", line 370, in begin
    return sel

In [8]:
airport_queue = queue.Queue()

def add_airport(graph, airport,region,country, continent, city, airport_queue):
    
    
    
    print('begin transaction', airport['airport'])
    try:
        create(graph, airport)
    except Exception as e:
        print('Error adding airport', airport['airport'])
        print(e)
    create(graph,Relationship(airport, "IN_REGION", region))
    create(graph,Relationship(airport, "IN_COUNTRY", country))
    create(graph,Relationship(airport, "IN_CONTINENT", continent))
    create(graph,Relationship(airport, "IN_CITY", city))
    
    airport_queue.put(airport)
    print("COUNTDOWN", latch.count)
    latch.countdown()
    #print("ADDED AIRPORT:", airport['airport'], airport['country'])
    
    
    print("end", latch.count)
    
    #threading.Thread(target=add_markets, args=(graph, airport, days))
    
  
    
def create_airport(row):
    print(row[3])
    region_name = row[9]
    country_name = row[8]
    continent_name = row[7]
    city_name = row[10]
    iata = row[13]

    region = r[region_name]
    country = c[country_name]
    continent = co[continent_name]
    city = ci[city_name]
    
    airport_name = row[3]
    airport = Node("Airport", airport=airport_name)
    airport['continent'] = continent_name
    airport['country'] = country_name
    airport['region'] = region_name
    airport['city'] = city_name
    airport['iata'] = iata
    
    print("Airport Done", airport_name)
    threading.Thread(target= add_airport, args= (graph, airport,region,country, continent, city, airport_queue)).start()
    
    


In [9]:
for row in AIRPORTS:
    create_airport(row)

Keflavik International Airport
Airport Done Keflavik International Airport
begin transaction Keflavik International Airport
Priština International Airport
Airport Done Priština International Airport
begin transaction Priština International Airport
Brussels Airport
Airport Done Brussels Airport
begin transaction Brussels Airport
Brussels South Charleroi Airport
Airport Done Brussels South Charleroi Airport
begin transaction Brussels South Charleroi Airport
Liège Airport
Airport Done Liège Airport
begin transaction Liège Airport
Berlin-Schönefeld International Airport
Airport Done Berlin-Schönefeld International Airport
begin transaction Berlin-Schönefeld International Airport
Dresden Airport
Airport Done Dresden Airport
begin transaction Dresden Airport
Frankfurt am Main International Airport
Airport Done Frankfurt am Main International Airport
begin transaction Frankfurt am Main International Airport
Münster Osnabrück Airport
Airport Done Münster Osnabrück Airport
begin transaction Mün

Exception in thread Thread-621:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-623:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-629:
Traceback (most recent call last):
  File "C:\Program Files\An

begin transactionGeorge Best Belfast City Airport
Airport Done George Best Belfast City Airport
 Belfast International Airport
begin transaction George Best Belfast City Airport
Birmingham International Airport
Airport Done Birmingham International Airport
begin transaction Birmingham International Airport
Manchester Airport
Airport Done Manchester Airport
begin transaction Manchester Airport
Robin Hood Doncaster Sheffield Airport
Airport Done Robin Hood Doncaster Sheffield Airport
begin transaction Robin Hood Doncaster Sheffield Airport
Cardiff International Airport
Airport Done Cardiff International Airport
begin transaction Cardiff International Airport
Bristol International Airport
Airport Done Bristol International Airport
begin transaction Bristol International Airport
Liverpool John Lennon Airport
Airport Done Liverpool John Lennon Airport
begin transaction Liverpool John Lennon Airport
London Luton Airport
Airport Done London Luton Airport
begin transaction London Luton Airport

Exception in thread Thread-685:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-687:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-690:
Traceback (most recent call last):
  File "C:\Program Files\An

begin transactionGlasgow International Airport
Airport Done Glasgow International Airport
 Aberdeen Dyce Airport
begin transaction Glasgow International Airport
Edinburgh Airport
Airport Done Edinburgh Airport
begin transaction Edinburgh Airport
Norwich International Airport
Airport Done Norwich International Airport
begin transaction Norwich International Airport
London Stansted Airport
Airport Done London Stansted Airport
begin transaction London Stansted Airport
Exeter International Airport
Airport Done Exeter International Airport
begin transaction Exeter International Airport
RAF Brize Norton
Airport Done RAF Brize Norton
begin transaction RAF Brize Norton
Amsterdam Airport Schiphol
Airport Done Amsterdam Airport Schiphol
begin transaction Amsterdam Airport Schiphol
Maastricht Aachen Airport
Airport Done Maastricht Aachen Airport
begin transaction Maastricht Aachen Airport
Eindhoven Airport
Airport Done Eindhoven Airport
begin transaction Eindhoven Airport
Cork Airport
Airport Don

...

Exception in thread Thread-858:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-861:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-864:
Traceback (most recent call last):
  File "C:\Program Files\An

begin transactionFalcone–Borsellino Airport
Airport Done Falcone–Borsellino Airport
 Catania-Fontanarossa Airport
begin transaction Falcone–Borsellino Airport
Cagliari Elmas Airport
Airport Done Cagliari Elmas Airport
begin transaction Cagliari Elmas Airport
Malpensa International Airport
Airport Done Malpensa International Airport
begin transaction Malpensa International Airport
Il Caravaggio International Airport
Airport Done Il Caravaggio International Airport
begin transaction Il Caravaggio International Airport
Turin Airport
Airport Done Turin Airport
begin transaction Turin Airport
Genoa Cristoforo Colombo Airport
Airport Done Genoa Cristoforo Colombo Airport
begin transaction Genoa Cristoforo Colombo Airport
Linate Airport
Airport Done Linate Airport
begin transaction Linate Airport
Cuneo International Airport
Airport Done Cuneo International Airport
begin transaction Cuneo International Airport
Bologna Guglielmo Marconi Airport
Airport Done Bologna Guglielmo Marconi Airport
beg

Exception in thread Thread-969:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-974:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-972:
Traceback (most recent call last):
  File "C:\Program Files\An

begin transactionSimferopol International Airport
Airport Done Simferopol International Airport
 Donetsk International Airport
begin transaction Simferopol International Airport
Kharkiv International Airport
Airport Done Kharkiv International Airport
begin transaction Kharkiv International Airport
Odessa International Airport
Airport Done Odessa International Airport
begin transaction Odessa International Airport
Pulkovo Airport
Airport Done Pulkovo Airport
begin transaction Pulkovo Airport
Minsk National Airport
Airport Done Minsk National Airport
begin transaction Minsk National Airport
Rostov-on-Don Airport
Airport Done Rostov-on-Don Airport
begin transaction Rostov-on-Don Airport
Sochi International Airport
Airport Done Sochi International Airport
begin transaction Sochi International Airport
Zhukovsky International Airport
Airport Done Zhukovsky International Airport
begin transaction Zhukovsky International Airport
Domodedovo International Airport
Airport Done Domodedovo Internat

Exception in thread Thread-1062:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-1065:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-f6f7f29958f7>", line 14, in add_airport
    create(graph,Relationship(airport, "IN_COUNTRY", country))
NameError: name 'country' is not defined

Exception in thread Thread-1068:
Traceback (most recent call last):
  File "C:\Program Files

In [ ]:
def add_airport_days():
    
    airport = airport_queue.get()
    
    airport_days = {}
    print("fffff",airport)
    
    for day in days:
        number = day['date']
        
        name = str(airport['iata'] + str(number) )
        print("gggg", name)
        airport_day = Node("AirportDay", name=name, airport = airport_name, day = number)
        
            
        airport_rel = Relationship( airport_day, "HAS_AIRPORT", airport)
        day_rel = Relationship( airport_day, "HAS_DAY", day)
        
        create(graph,airport_day)
        create(graph,airport_rel)
        create(graph,day_rel)

        airport_days[number] = (airport_day)
        
        early, late = None
        for airport_day_index in list(range(NO_OF_DAYS-1))[::-1]:
            early = airport_days[airport_day_index]
            late =  airport_days[(airport_day_index +1) % NO_OF_DAYS]

            next_market = Relationship(early, "NEXT_MARKET", late)
            create(graph, next_market)


while not airport_queue.empty():
    print("New Thingy")
    threading.Thread(target=add_airport_days)
    
    
    
        
    
        
   
    
    

In [ ]:
while not airport_queue.empty():
    
    print(airport_queue.get())

In [ ]:
print(sorted(regions))